<a href="https://www.comet.com/site/?utm_medium=colab&utm_source=comet-examples&utm_campaign=unsloth" >
  <img src="https://cdn.comet.ml/img/notebook_logo.png">
</a>

# Comet and unsloth

[Comet](https://www.comet.com/site/?utm_medium=colab&utm_source=comet-examples&utm_campaign=unsloth) is an MLOps platform designed to help data scientists and teams build better models faster! Comet provides tooling to track, explain, manage, and monitor your models in a single place! It works with Jupyter notebooks and scripts and-- most importantly--it's 100% free to get started!

[unsloth](https://github.com/unslothai/unsloth) dramatically improves the speed and efficiency of LLM fine-tuning for models including Llama, Phi-3, Gemma, Mistral, and more. For a full listed of 100+ supported unsloth models, [see here](https://huggingface.co/unsloth).

Instrument your torchtune training runs with Comet to start managing experiments with efficiency, reproducibility, and collaboration in mind.

Find more information about [our integration with torchtune here](https://www.comet.com/docs/v2/integrations/third-party-tools/unsloth?utm_medium=colab&utm_source=comet-examples&utm_campaign=unsloth) or [learn about our other integrations here](https://www.comet.com/docs/v2/integrations?utm_medium=colab&utm_source=comet-examples&utm_campaign=unsloth).

## ⚙ Install and import dependencies

In [ ]:
%pip install comet_ml "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" "torch>=2.4.0" xformers trl peft accelerate bitsandbytes triton

In [ ]:
import comet_ml

comet_ml.login()
exp = comet_ml.Experiment(project_name="comet-example-unsloth")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## ⚙ Download model

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
load_in_4bit = True

Find the full list of [100+ supported unsloth models here](https://huggingface.co/unsloth). For a full list of supported 4-bit models see [here](https://huggingface.co/collections/unsloth/load-4bit-models-4x-faster-659042e3a41c3cbad582e734)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

## ⚙ Add LoRA adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # rank stabilized LoRA
    loftq_config=None,  # LoftQ
)

## ⚙ Data preparation

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # add EOS_TOKEN


def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # add EOS_TOKEN, otherwise your generation will go on forever
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }


pass

from datasets import load_dataset

dataset = load_dataset("yahma/alpaca-cleaned", split="train")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

## ⚙ Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
exp.end()

## ⚙ Inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Continue the fibonnaci sequence.",  # instruction
            "1, 1, 2, 3, 5, 8",  # input
            "",  # output - leave this blank for generation
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
tokenizer.batch_decode(outputs)